In [76]:
import glob
import numpy as np
import scipy as sp
import einops as eo
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import magpylib as magpy
import os


M0 = 1440 #mT
shape = [3*25.4/16, 25.4/8] #radius and height
x0 = np.array([0,0,10, 0,0,1])

In [77]:
def B_dipole(position, rotation, M0, shape):
    R = np.sqrt(np.sum(position**2, axis=1))
    B = (M0 * (shape[0]) ** 2 * shape[1] / (16)) * (
        (
            3
            * position
            / R[:, np.newaxis] ** 5
            * (eo.einsum(position, rotation, "sensor dim,  dim -> sensor"))[
                :, np.newaxis
            ]
        )
        - rotation[np.newaxis, :] / (R[:, np.newaxis] ** 3)
    )
    return B


def getField_dipole(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def getField_dipole_fixed(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def cost_dipole(x, B, positions, M0, shape):
    diff = getField_dipole(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)


def getField_cylinder(x, positions, M0, shape):
    B=magpy.getB(
        sources="Cylinder",
        position=x[:3],
        orientation=Rotation.align_vectors(x[3:], np.array([0, 0, 1]))[0],
        observers=positions,
        dimension=shape,
        polarization=(0, 0, M0),
    )
    return B

def cost_cylinder(x, B, positions, M0, shape):
    diff = getField_cylinder(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)

def minimize(x0, B, positions, M0, shape, *args):
    #print("Starting mimimization")
    b_args = (B, positions, M0, shape)
    cons = [{"type": "eq", "fun": lambda x: x[3] ** 2 + x[4] ** 2 + x[5] ** 2 - 1}]
    bounds = [(-100, 100), (-100, 100), (0, 100), (-1, 1), (-1, 1), (-1, 1)]
    res = sp.optimize.minimize(
        fun=cost_dipole, x0=x0, args=b_args, tol=1e-100, constraints=cons, bounds=bounds, *args
    ).x  
    #print(f"Finished mimimization with shape {b_args[3]} at {res}")
    return res

def circle_radius(x1, y1, x2, y2, x3, y3):
    # Calculate lengths of sides of the triangle formed by the three points
    a = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    b = math.sqrt((x3 - x2) ** 2 + (y3 - y2) ** 2)
    c = math.sqrt((x3 - x1) ** 2 + (y3 - y1) ** 2)

    # Calculate the semi-perimeter of the triangle
    s = (a + b + c) / 2

    # Calculate the area of the triangle using Heron's formula
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))

    # Calculate the radius of the circle using the formula: radius = (abc) / (4 * area)
    radius = (a * b * c) / (4 * area)
    
    return radius

def comparison_plot(locact, locpred, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(index, pred, label='Predicted' + rel)
    plt.scatter(index, act, label='Actual' + rel)
    plt.xlabel('Index')
    plt.ylabel(axis + '-component')
    plt.title('Comparison of '+axis+ ' Location' + rel+ ': Predicted vs Actual')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def comp_plot3D(locact, locpred, xact, yact, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    fig = go.Figure(data=[
        go.Scatter3d(x=xact, y=yact, z=pred, name='Predicted' + rel),
        go.Scatter3d(x=xact, y=yact, z=act, name='Actual' + rel)])
    fig.show()

    return

def displacement_plot(actual, predicted):

    dispx = np.array(predicted[:,0]-predicted[:,0][0])
    dispy = np.array(predicted[:,1]-predicted[:,1][0])
    dispz = np.array(predicted[:,2]-predicted[:,2][0])
    disp = np.sqrt(dispx**2+dispy**2+dispz**2)
    actdispx = np.array(actual[:,0]-actual[:,0][0])
    actdispy = np.array(actual[:,1]-actual[:,1][0])
    actdispz = np.array(actual[:,2]-actual[:,2][0])
    actdisp = np.sqrt(actdispx**2+actdispy**2+actdispz**2)

    plt.figure(figsize=(8, 6))
    plt.scatter(list(range(len(disp))), disp, label='Predicted Displacement')
    plt.scatter(list(range(len(actdisp))), actdisp, label='Actual Displacement')
    plt.xlabel('Index')
    plt.ylabel('Displacement (mm)')
    plt.title('Magnet Displacement')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def importfitting(folder, indices):

    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    print(len(file_list))
    data = np.load(file_list[0])
    pos = data['pos'][0]

    locpred = [np.array([0,0,0])]
    anglepred = [np.array([0,0,0])]
    Bstdev = [np.array(pos)]
    for x in range(len(file_list)):
        data = np.load(file_list[x])
        mags = data['mags']
        pos = data['pos'][0][indices]
        B = np.mean(mags[0][indices], axis=0)
        mags_stdev = np.std(mags, axis=0)
        x_res = minimize(x0, B, pos, M0, shape)
        locpred = np.append(locpred, [x_res[0:3]], axis=0)
        anglepred = np.append(anglepred, [x_res[3:6]], axis=0)
        Bstdev = np.append(Bstdev, [mags_stdev], axis=0)
    locpred = np.delete(locpred, 0, axis=0)
    anglepred = np.delete(anglepred, 0, axis=0)
    Bstdev = np.delete(Bstdev, 0 ,axis=0)

    return pos, mags, locpred, anglepred, Bstdev

def actualangle(xind, yind, xscal, xcon, yscal, ycon, isoff):

    angact = [np.array([0, 0, 0])]
    for y in range(yind):
        for x in range(xind):
            R = Rotation.from_euler('ZYX' ,np.array([0, yscal*y-ycon, xscal*x-xcon]), degrees=True)
            norm = np.array([[0, 0, 1]])
            rotated = R.apply(norm, inverse=False)[0]
            #add = np.array([np.sin(np.deg2rad(2*y-4)), -np.sin(np.deg2rad(2*x-4)), np.cos(np.deg2rad(2*y-4))*np.cos(np.deg2rad(2*x-4))])
            angact = np.append(angact, [rotated], axis=0)
    angact = np.delete(angact, 0, axis=0)
    locact = angact*isoff

    return angact, locact

In [78]:
folder = os.path.join('..', './MLX07102024_D32N52_AboutZ_10mm2')
pos, mags, locpred, anglepred, Bstdev = importfitting(folder, range(16))
np.append(locpred, anglepred, axis=1)

10


array([[ 3.00683067e+00, -9.32528827e+00,  1.87631790e+01,
         3.56595681e-02, -2.26189668e-01,  9.73430341e-01],
       [ 9.48459872e+00, -4.53299799e+00,  1.92681546e+01,
         2.15434814e-01, -6.02038570e-02,  9.74660626e-01],
       [ 1.23096342e+01,  9.05765484e-01,  1.92989884e+01,
         3.24164256e-01,  8.45995984e-03,  9.45962982e-01],
       [ 7.71528151e+00,  7.33711165e+00,  2.02061856e+01,
         1.35433622e-01,  1.23809980e-01,  9.83020256e-01],
       [ 1.33529063e+00,  1.15282956e+01,  1.96737984e+01,
         1.25065826e-02,  2.88166313e-01,  9.57498700e-01],
       [-4.30846003e+00,  8.66784609e+00,  1.97785855e+01,
        -5.47496124e-02,  1.80989064e-01,  9.81959999e-01],
       [-8.73289366e+00,  2.51706236e+00,  1.93040905e+01,
        -1.97912938e-01,  2.78291598e-02,  9.79824478e-01],
       [-7.75164148e+00, -3.08191644e+00,  1.90568398e+01,
        -1.68092738e-01, -3.80964860e-02,  9.85034765e-01],
       [-2.67785227e+00, -8.36522560e+00,  1.888

In [79]:
file_list = sorted(glob.glob(f'{folder}/*.npz'))
len(file_list)
data = np.load(file_list[0])
pos = data['pos'][0]

all_mags = [np.array([0,0,0])]
for x in range(len(file_list)):
    data = np.load(file_list[x])
    mags = data['mags']
    all_mags = np.append(all_mags, mags[0], axis=0)
    pos = data['pos'][0]
    B = np.mean(mags, axis=0)
    mags_stdev = np.std(mags, axis=0)
all_mags = np.delete(all_mags, 0, axis=0)

print(all_mags)
bx = all_mags[:,0]
by = all_mags[:,1]
bz = all_mags[:,2]

print(bx)
len(bx)

[[-1.056300e+00 -3.741000e-01  3.276196e+00]
 [ 4.956000e-01 -3.924000e-01  3.643552e+00]
 [ 1.489500e+00 -2.967000e-01  2.340624e+00]
 [ 1.311000e+00 -1.299000e-01  1.023176e+00]
 [-7.971000e-01 -1.359900e+00  2.249632e+00]
 [ 3.414000e-01 -1.490100e+00  2.410804e+00]
 [ 1.123200e+00 -1.161300e+00  1.728848e+00]
 [ 1.075800e+00 -6.171000e-01  7.632680e-01]
 [-4.875000e-01 -1.431300e+00  1.052216e+00]
 [ 1.536000e-01 -1.462500e+00  1.163536e+00]
 [ 6.327000e-01 -1.129500e+00  8.397400e-01]
 [ 6.534000e-01 -7.191000e-01  4.322120e-01]
 [-2.634000e-01 -9.372000e-01  3.538040e-01]
 [ 1.410000e-02 -9.729000e-01  3.997840e-01]
 [ 2.958000e-01 -8.682000e-01  2.947560e-01]
 [ 3.549000e-01 -5.820000e-01  1.427800e-01]
 [ 5.208000e-01  8.082000e-01  3.136320e+00]
 [ 1.312200e+00  5.265000e-01  2.015376e+00]
 [ 1.226700e+00  2.916000e-01  8.852360e-01]
 [ 7.854000e-01  1.449000e-01  3.054040e-01]
 [ 5.421000e-01 -6.417000e-01  3.295072e+00]
 [ 1.362900e+00 -4.836000e-01  2.057000e+00]
 [ 1.27710

160

In [91]:
bx7 = []
by7 = []
bz7 = []
for x in range(160):
    if x % 16 == 6:
        bx7.append(bx[x])
        by7.append(by[x])
        bz7.append(bz[x])

In [81]:
index = np.array(range(160))
measuredbx = go.Scatter(x=index, y=bx7, mode='markers')
layoutbx = go.Layout(title='b_x over time')
figbx = go.Figure(data=[measuredbx], layout=layoutbx)
figbx.update_yaxes(scaleanchor='y')
figbx.update_layout(
    xaxis_title="Index",
    yaxis_title="b_x [mT]"
)
figbx.show()

In [82]:
measuredby = go.Scatter(x=index, y=by7, mode='markers')
layoutby = go.Layout(title='b_y over time')
figby = go.Figure(data=[measuredby], layout=layoutby)
figby.update_yaxes(scaleanchor='y')
figby.update_layout(
    xaxis_title="Index",
    yaxis_title="b_y [mT]"
)
figby.show()

In [83]:
measuredbz = go.Scatter(x=index, y=bz7, mode='markers')
layoutbz = go.Layout(title='b_z over time')
figbz = go.Figure(data=[measuredbz], layout=layoutbz)
figbz.update_yaxes(scaleanchor='y')
figbz.update_layout(
    xaxis_title="Index",
    yaxis_title="b_z [mT]"
)
figbz.show()

In [84]:
size = 2

measured = go.Cone(x=locpred[:,0], y=locpred[:,1], z=locpred[:,2], u=anglepred[:,0], v=anglepred[:,1], w=anglepred[:,2], 
                             sizemode="absolute", sizeref=size, )

#calculated = go.Cone(x=locact[:,0], y=locact[:,1], z=locact[:,2], u=actang[:,0], v=actang[:,1], w=actang[:,2], 
#                             sizemode="absolute", sizeref=size, colorscale=[(0, 'blue'), (1, 'blue')])

layout = go.Layout(title='Location Prediction', scene=dict(aspectmode='data', xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))

fig = go.Figure(data=[measured], layout=layout)

fig.show()

In [85]:
a = 0
b = 1

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=locpred[:,2], mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='Rotation over Time', scene=dict(xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [86]:
a = 0
b = 2

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=np.array(range(len(locpred[:,a]))), mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Colorbar"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='x vs z', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]', zaxis_title='Index'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [87]:
a = 1
b = 2

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=np.array(range(len(locpred[:,a]))), mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='y vs z', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]', zaxis_title='Index'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [88]:
a = 0
b = 1

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs y (measured)')
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="y [mm]"
)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [89]:
a = 0
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs z (measured)', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="z [mm]"
)
fig2.show()

In [90]:
a = 1
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='y vs z (measured)', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="y [mm]",
    yaxis_title="z [mm]"
)
fig2.show()

Measure natural background (should be mostly b_z)
Try measurement in different location
Find optics plate to secure stage to base
use magpylib to generate expected values 